In [ ]:
%run ../ms_packages_and_functions.ipynb

In [ ]:
output_dir = pjoin(os.getcwd(), 'processed_data')
data_dir  = pjoin(os.getcwd(), 'raw_data')
fig_dir   = pjoin(os.getcwd(), 'figures')

In [ ]:
rand_state = '002'#np.random.randint(1000000000)

In [ ]:
folder_path = pjoin(output_dir,'rj1031209-UMAP-embedding-{}'.format(rand_state))

if not os.path.exists(folder_path):
    os.mkdir(folder_path)
    print("created directory: {}".format(folder_path))
else:
    print("All files will be saved in: {}".format(folder_path))

In [ ]:
fig_dir = pjoin(folder_path,'figures')

if not os.path.exists(fig_dir):
    os.mkdir(fig_dir)
    print("created directory: {}".format(fig_dir))
else:
    print("All files will be saved in: {}".format(fig_dir))

In [ ]:
data_FNs = loadPickle('processed_data/rj1031209_CenterOut_FN_10msBin_with500buffer_MIonly.pkl')

In [ ]:
trials = loadPickle('processed_data/rj031209_trials_by_cutoff.pkl')
cutoff = 1.5

# Flatten FNs

In [ ]:
cutoff = 1.5

FN_flattened = {}
FN_flattened['data'] = []
FN_flattened['FN_info'] = []

for direction in range(0,len(data_FNs)):
    tr = -1
    for trial in range(0,len(data_FNs[direction])):
        
        if trial in trials[cutoff]['trials'][direction]:
            tr += 1
            for time in range(0,len(data_FNs[direction][trial])):
                status('flattening FNs: {},{},{}'.format(direction,trial,time))
                FN_flattened['data'].append(flattenNetwork(data_FNs[direction][trial][time]))
                FN_flattened['FN_info'].append([direction,tr,time])
            
save_dir = pjoin(output_dir,f'rj1031209_CenterOut_FN-Flattened_10msBin_with500buffer_MIonly-{cutoff}-trialCount.pkl')
savePickle(save_dir,FN_flattened)
del data_FNs

# Embed

In [ ]:
rand_state = np.random.randint(1000000000)
umap_reducer = umap.UMAP(n_components=2, n_neighbors=100, min_dist=0.1,random_state=rand_state,metric='cosine')

start_time = time2bin(0)
end_time = time2bin(2)
node_idx = [i for i,x in enumerate(FN_flattened['FN_info']) if (x[2] >= start_time
                                                                and x[2] < end_time) ]

embeddings = umap_reducer.fit_transform(np.array(FN_flattened['data'])[node_idx])
save_dir = pjoin(folder_path,'FN_UMAP_embedding.pkl')
savePickle(save_dir,embeddings)

In [ ]:
figure = plt.figure(figsize=(15,15))
ax = figure.add_subplot(111)

cmap = sns.color_palette("husl",8)
temp_colors = [cmap[x[0]] for x in FN_flattened['FN_info']]
alpha_values= np.array([x[2] for x in FN_flattened['FN_info']])
alpha_values = alpha_values/np.max(alpha_values)

start_time = time2bin(0)
end_time = time2bin(2)

node_idx = [i for i,x in enumerate(FN_flattened['FN_info']) if (x[2] >= start_time
                                                                and x[2] < end_time) ]
colors = np.array([list(x[0])+[x[1]] for x in zip(temp_colors,alpha_values)])[node_idx]

# colors = np.array(temp_colors)[node_idx]
# colors = [cmap[FN_flattened['FN_info'][x][0]] for x in node_idx]
ax.scatter(embeddings[:, 0], embeddings[:, 1],c=colors,marker='.')
ax.axis('equal')
ax.set_axis_off()
# plt.show()
plt.savefig(pjoin(folder_path,'UMAP-embedding-nopretrial.png'))

In [ ]:
figure = plt.figure(figsize=(15,15))
ax = figure.add_subplot(111)

cmap = sns.color_palette("husl",8)
temp_colors = [cmap[x[0]] for x in FN_flattened['FN_info']]
alpha_values= np.array([x[2] for x in FN_flattened['FN_info']])
alpha_values = alpha_values/np.max(alpha_values)

start_time = time2bin(-0.5)
end_time = time2bin(1.5)

node_idx = [i for i,x in enumerate(FN_flattened['FN_info']) if (x[2] >= start_time
                                                                and x[2] < end_time) ]
colors = np.array([list(x[0])+[x[1]] for x in zip(temp_colors,alpha_values)])[node_idx]

# colors = np.array(temp_colors)[node_idx]
# colors = [cmap[FN_flattened['FN_info'][x][0]] for x in node_idx]
ax.scatter(embeddings[node_idx, 0], embeddings[node_idx, 1],c=colors,marker='.')
ax.axis('equal')
ax.set_axis_off()
# plt.show()
plt.savefig(pjoin(folder_path,'UMAP-embedding-1-5sec-alpha.png'))

In [ ]:
times = np.arange(-0.5,2.5,0.5)

for t in times:
    figure = plt.figure(figsize=(15,15))

    ax = figure.add_subplot(111)

    start_time = time2bin(t)
    end_time = time2bin(t+0.5)
    node_idx = [i for i,x in enumerate(FN_flattened['FN_info']) if (x[2] >= start_time
                                                                    and x[2] < end_time) ]

    gray_idx = [i for i,x in enumerate(FN_flattened['FN_info']) if (x[2] < start_time)]        
    cmap = sns.color_palette("husl",8)
    colors = [cmap[FN_flattened['FN_info'][x][0]] for x in node_idx]


    if gray_idx:
        ax.scatter(embeddings[gray_idx, 0], embeddings[gray_idx, 1],c='xkcd:silver',alpha = 0.1, marker='.')

    ax.scatter(embeddings[node_idx, 0], embeddings[node_idx, 1],c=colors,marker='.')

    ax.axis('equal')
    ax.axis('off')
    plt.savefig(pjoin(folder_path,'UMAP-emebedding-{}.png'.format(int(t*1000))))

# Graph Alignment Score

In [ ]:
data_alignment_scores = []

min_trial_length = min([len(x) for y in data_FNs for x in y])
all_trials = [[dir,trial] for dir in range(0,len(data_FNs))
                        for trial in range(0,len(data_FNs[dir]))]
FN_pairs = [y for y in itertools.combinations(all_trials,2)]

cdist = [circdistance(8,y[0][0],y[1][0]) for y in FN_pairs]

for time in range(0,min_trial_length):
    status("> getting alignment scores for time: {}s".format(bin2time(time)))
    data_alignment_scores.append([])
    [data_alignment_scores[time].append([]) for x in np.unique(cdist)]

    for c, pair in zip(cdist,FN_pairs):
        score = align_score(data_FNs[pair[0][0]][pair[0][1]][time],data_FNs[pair[1][0]][pair[1][1]][time])
        data_alignment_scores[time][c].append(score)

In [ ]:
min_trial_length = 200

In [ ]:
figure = plt.figure(figsize=(10,20))
plt.suptitle('Alignment Scores: Data',fontsize=14,fontweight='bold')
bins = np.arange(-0.05,0.5,0.01)
dist_vals= [0,1,4]
cmap = sns.color_palette("viridis", 5)

labels = ['same','neighboring','opposite']
sig_test = []
ticks = []
time_plot = 1
for t in range(0,min_trial_length,time_plot):
    # plot alignment scores from Data

    # data dist
    for i, dist in enumerate(dist_vals):
        y,x = np.histogram(data_alignment_scores[t][dist],bins=bins)
        y = y/len(data_alignment_scores[t][dist])
        y = (y*1000)-(10*t)
        plt.plot(x[1:],y,color=cmap[dist],label=labels[i])

    ticks.append(y[0])

    sig = ''
    for i in itertools.combinations(dist_vals,2):
        ks_stat = stats.ks_2samp(data_alignment_scores[t][i[0]],data_alignment_scores[t][i[1]])

        if i[0]==0 and i[1]==1:
            if ks_stat[1]<0.001:
                sig +='*'
                print('{} vs {}: significant at {},pval = {},'.format(i[0],i[1],bin2time(t,lastBin=True),ks_stat[1]))
            else:
                sig +='  '

        if i[0]==0 and i[1]==4 and ks_stat[1]<0.001:
            sig += ' *'
            print('{} vs {}: significant at {},pval = {},'.format(i[0],i[1],bin2time(t,lastBin=True),ks_stat[1]))

    sig_test.append(sig)

# plot a line at 0
plt.plot([0,0],[150,-10*min_trial_length,],'k--')

#formatting
ax = plt.gca()
ax.tick_params(axis=u'both', which=u'both',length=0)
ax.set_ylabel('Time (s)')
ax.set_xlabel('Alignment Score')
ax.set_yticklabels([round(bin2time(x,lastBin=True),3) for x in np.arange(0,min_trial_length,time_plot)])
ax.set_yticks(ticks)#[-10*t for t in np.arange(0,min_trial_length,time_plot)])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

ax2 = ax.twinx()
ax2.set_ylim(ax.get_ylim())
ax2.tick_params(axis=u'both', which=u'both',length=0)
ax2.set_ylabel('significance (* if p<0.001')
ax2.set_xlabel('Alignment Score')
ax2.set_yticklabels(sig_test)
ax2.set_yticks(ticks)#[-10*t for t in np.arange(0,min_trial_length,time_plot)])
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.spines['bottom'].set_visible(False)
ax2.spines['left'].set_visible(False)

plt.tight_layout()
plt.savefig(pjoin(folder_path,'align-score-data-by{}bins.png'.format(time_plot)))

In [ ]:
plt.figure(figsize=(14,5))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
# plt.suptitle(title[i],fontsize=14,fontweight='bold')
time_label = [round(bin2time(x,lastBin=True),3) for x in np.arange(0,min_trial_length,time_plot)]
for dist in range(0,5):#dist_vals:
    y = np.array([np.mean(data_alignment_scores[t][dist]) for t in range(0,min_trial_length,time_plot)])
    ax1.plot(time_label, y, color=cmap[dist])
    ax1.plot([0,0],[0,0.35],'k--',alpha=0.5)
    ax1.plot([1,1],[0,0.35],'k--',alpha=0.5)
    ax1.set_xlabel('Time (s)')
    ax1.set_title('Mean Alignment Score')
    ax1.set_title('Mean',fontsize=12,fontweight='bold')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['bottom'].set_visible(False)
    ax1.spines['left'].set_visible(False)
    
    var = np.array([np.var(data_alignment_scores[t][dist]) for t in range(0,min_trial_length,time_plot)])
    ax2.plot(time_label,var,color=cmap[dist])
    ax2.plot([0,0],[0,0.0016],'k--',alpha=0.5)
    ax2.plot([1,1],[0,0.0016],'k--',alpha=0.5)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Alignment Score Variance')
    ax2.set_title('Variance',fontsize=12,fontweight='bold')
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_visible(False)
    ax2.spines['left'].set_visible(False)
plt.savefig(pjoin(folder_path,'mean-and-variance-GAS.png'))

# Reciprocity

In [ ]:
p = 80

cmap = sns.color_palette('husl',8)
plt.figure(figsize=(10,5))
for direction in range(0,len(data_FNs)):
    
    all_reciprocity = []
    for trial in range(0,len(data_FNs[direction])):
        reciprocity = []
        for time in range(0,len(data_FNs[direction][trial])):
            thresholded_FN = thresholdNetwork(data_FNs[direction][trial][time],p)
            G = nx.from_numpy_array(thresholded_FN,create_using=nx.DiGraph())
            reciprocity.append(nx.algorithms.overall_reciprocity(G))
        
        all_reciprocity.append(reciprocity)
    min_trials = np.min([len(x) for x in all_reciprocity])
    x = [bin2time(x,lastBin=True) for x in np.arange(0,len(data_FNs[direction][trial]))]
    x = x[0:min_trials]
    all_reciprocity = [x[:min_trials] for x in all_reciprocity]
    mean = np.mean(all_reciprocity,axis=0)
    std = np.std(all_reciprocity,axis=0)
    plt.plot(x, mean,color=cmap[direction],alpha=0.2)
    plt.fill_between(x,mean-std,mean+std,color=cmap[direction],alpha=0.2)
    plt.plot([0,0],[0,1],'k--',alpha=0.5)     
    plt.plot([1,1],[0,1],'k--',alpha=0.5) 
    plt.title('Reciprocity for Direction {}, at {} percentile'.format(direction,p))
    plt.xlabel('time(s)')
    plt.ylabel('Reciprocity')
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    plt.savefig(pjoin(folder_path,'reciprocity-vs-time_p{}.png').format(p))

In [ ]:
num_trials = [len(x) for x in data_FNs]
plt.bar(np.arange(0,8),num_trials)

In [ ]:
np.arange(0,360,45)

In [ ]:
data_FNs[0][0][0].shape